In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [2]:
import tensorflow
print(tensorflow.__version__)
import tensorflow.keras as keras

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, \
  Dense, Activation, Dropout, GlobalMaxPooling1D, Add, Flatten

import numpy as np
from sklearn.model_selection import StratifiedKFold

2.1.0-rc1


In [0]:
# Parameters:
# Word Embedding
max_features = 1000
maxlen = 400
embedding_dims = 250

# Convolution
kernel_size = 3 
filters = 250 
pool_size = 4 

# Training
batch_size = 30
epochs = 5

In [4]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
x_data = np.concatenate((x_train, x_test))
y_data = np.concatenate((y_train, y_test))
skf = StratifiedKFold(n_splits=5, random_state=42)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)


In [0]:
# Network Architecture
def create_DPCNN_model():
  # Deep Pyramid Convolutional Neural Networks for Text Categorization
  # paper's "Region Embedding" is just an extra conv layer?
  var_input = Input(shape=(400,))
  emb_layer = Embedding(max_features,
                      embedding_dims,
                      input_length=maxlen)(var_input)
  #text_embed = SpatialDropout1D(0.2)(emb_layer)

  # First block
  conv_layer_1 = Conv1D(filters, kernel_size=kernel_size, padding='same',activation='relu',
                  strides=1)(emb_layer)
  conv_layer_2 = Conv1D(filters, kernel_size=kernel_size, padding='same',activation='relu',
                  strides=1)(conv_layer_1)
  shortcut_1 = Add()([conv_layer_2, emb_layer])
  maxpooling_1 = MaxPooling1D(pool_size=3, strides=2)(shortcut_1)

  #Block2
  conv_layer_3 = Conv1D(filters, kernel_size=kernel_size, padding='same',activation='relu',
                  strides=1)(maxpooling_1)
  conv_layer_4 = Conv1D(filters, kernel_size=kernel_size, padding='same',activation='relu',
                  strides=1)(conv_layer_3)
  shortcut_2 = Add()([conv_layer_4, maxpooling_1])
  maxpooling_2 = MaxPooling1D(pool_size=3, strides=2)(shortcut_2)

  # Block 3
  conv_layer_5 = Conv1D(filters, kernel_size=kernel_size, padding='same',activation='relu',
                  strides=1)(maxpooling_2)
  conv_layer_6 = Conv1D(filters, kernel_size=kernel_size, padding='same',activation='relu',
                  strides=1)(conv_layer_5)
  shortcut_3 = Add()([conv_layer_6, maxpooling_2])
  maxpooling_3 = MaxPooling1D(pool_size=3, strides=2)(shortcut_3)

  flatten = Flatten()(maxpooling_3)

  dense_layer = Dense(1, activation='sigmoid')(flatten)
  model = Model(inputs=var_input, outputs=dense_layer)
  model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  return model

In [6]:
create_DPCNN_model().summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 400)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 400, 250)     250000      input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 400, 250)     187750      embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 400, 250)     187750      conv1d[0][0]                     
______________________________________________________________________________________________

In [7]:
for train, test in skf.split(x_data, y_data):
  target = create_DPCNN_model()
  target.fit(x_data[train], y_data[train],
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_data[test], y_data[test]))

train data shape: (40000,)
test data shape: (10000,)
Train on 40000 samples, validate on 10000 samples
Epoch 1/5
40000/40000 [==============================] - 41s 1ms/sample - loss: 0.3746 - accuracy: 0.8209 - val_loss: 0.3086 - val_accuracy: 0.8650
Epoch 2/5
40000/40000 [==============================] - 33s 832us/sample - loss: 0.2677 - accuracy: 0.8895 - val_loss: 0.2813 - val_accuracy: 0.8832
Epoch 3/5
40000/40000 [==============================] - 33s 834us/sample - loss: 0.2172 - accuracy: 0.9114 - val_loss: 0.2794 - val_accuracy: 0.8861
Epoch 4/5
40000/40000 [==============================] - 34s 844us/sample - loss: 0.1638 - accuracy: 0.9370 - val_loss: 0.3204 - val_accuracy: 0.8774
Epoch 5/5
40000/40000 [==============================] - 33s 833us/sample - loss: 0.1115 - accuracy: 0.9598 - val_loss: 0.3455 - val_accuracy: 0.8781
train data shape: (40000,)
test data shape: (10000,)
Train on 40000 samples, validate on 10000 samples
Epoch 1/5
40000/40000 [=======================

In [0]:
# Network Architecture
def create_DPCNN_SELU():
  # DPCNN w SELU activation
  var_input = Input(shape=(400,))
  emb_layer = Embedding(max_features,
                      embedding_dims,
                      input_length=maxlen)(var_input)
  #text_embed = SpatialDropout1D(0.2)(emb_layer)

  # First block
  conv_layer_1 = Conv1D(filters, kernel_size=kernel_size, padding='same',activation='selu',
                  strides=1)(emb_layer)
  conv_layer_2 = Conv1D(filters, kernel_size=kernel_size, padding='same',activation='selu',
                  strides=1)(conv_layer_1)
  shortcut_1 = Add()([conv_layer_2, emb_layer])
  maxpooling_1 = MaxPooling1D(pool_size=3, strides=2)(shortcut_1)

  #Block2
  conv_layer_3 = Conv1D(filters, kernel_size=kernel_size, padding='same',activation='selu',
                  strides=1)(maxpooling_1)
  conv_layer_4 = Conv1D(filters, kernel_size=kernel_size, padding='same',activation='selu',
                  strides=1)(conv_layer_3)
  shortcut_2 = Add()([conv_layer_4, maxpooling_1])
  maxpooling_2 = MaxPooling1D(pool_size=3, strides=2)(shortcut_2)

  # Block 3
  conv_layer_5 = Conv1D(filters, kernel_size=kernel_size, padding='same',activation='selu',
                  strides=1)(maxpooling_2)
  conv_layer_6 = Conv1D(filters, kernel_size=kernel_size, padding='same',activation='selu',
                  strides=1)(conv_layer_5)
  shortcut_3 = Add()([conv_layer_6, maxpooling_2])
  maxpooling_3 = MaxPooling1D(pool_size=3, strides=2)(shortcut_3)

  flatten = Flatten()(maxpooling_3)

  dense_layer = Dense(1, activation='sigmoid')(flatten)
  model = Model(inputs=var_input, outputs=dense_layer)
  model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  return model

In [9]:
for train, test in skf.split(x_data, y_data):
  target = create_DPCNN_SELU()
  target.fit(x_data[train], y_data[train],
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_data[test], y_data[test]))

train data shape: (40000,)
test data shape: (10000,)
Train on 40000 samples, validate on 10000 samples
Epoch 1/5
40000/40000 [==============================] - 34s 844us/sample - loss: 0.5138 - accuracy: 0.7688 - val_loss: 0.3780 - val_accuracy: 0.8470
Epoch 2/5
40000/40000 [==============================] - 33s 814us/sample - loss: 1.6287 - accuracy: 0.8185 - val_loss: 1.5625 - val_accuracy: 0.6587
Epoch 3/5
40000/40000 [==============================] - 33s 813us/sample - loss: 0.5547 - accuracy: 0.8409 - val_loss: 0.5131 - val_accuracy: 0.8416
Epoch 4/5
40000/40000 [==============================] - 33s 813us/sample - loss: 0.5464 - accuracy: 0.8518 - val_loss: 0.3579 - val_accuracy: 0.8725
Epoch 5/5
40000/40000 [==============================] - 32s 811us/sample - loss: 0.2717 - accuracy: 0.8923 - val_loss: 0.3384 - val_accuracy: 0.8662
train data shape: (40000,)
test data shape: (10000,)
Train on 40000 samples, validate on 10000 samples
Epoch 1/5
40000/40000 [=====================